# Migration 4.01 Hierarchy

Migration 4.01 uses the "Country of Residence Groupings" outlined in the supporting info spreadsheet
'internalmigrationtableofcontents.xls'. This is what we've used to create the countries
hierarchy.csv file.

It's possibe the name may change, the table itself is linked to from the 'Contents' tab of the
spreadsheets we're transforming.

The "Country of Residence Groupings" has numerous caveats but it won't be necessary to rebuild it every time
(a change in grouping means a new edition and a new hierarchy but countries don't change groups every year).

There's no way to forsee what future caveats will need to accomodated so this script may just be
a starting point the next time we need to build a hierarchy.csv file for migration 4.01.


In [64]:
# imports
from databaker.framework import *
import pandas as pd

# simple Dict to hold everything in until we're ready to switch to a dataframe
dictForFrame = {
    "Label":[],
    "Parent":[]
}

"""
IMPORTANT

the following switch, when True, will include the additional groupinds (EEA, EFTA, Commonwealth).
be aware these are regorupings of children that already appear in other more conventional groups.
"""
includeAdditionals = True


## Build The Higher Level Relationships

We'start by building the higher level groups that these groups will sit under, and connect them all up.

Will make this explicit as there aren't that many of them.


In [65]:
# Parent
dictForFrame["Label"].append("All countries of last or next residence")
dictForFrame["Parent"].append("")

# EU
dictForFrame["Label"].append("European Union")
dictForFrame["Parent"].append("All countries of last or next residence")

dictForFrame["Label"].append("European Union EU15")
dictForFrame["Parent"].append("European Union")
dictForFrame["Label"].append("European Union EU8")
dictForFrame["Parent"].append("European Union")
dictForFrame["Label"].append("European Union EU2")
dictForFrame["Parent"].append("European Union")
dictForFrame["Label"].append("Other European Union")
dictForFrame["Parent"].append("European Union")


# Non Eu - Europe exc European Union
dictForFrame["Label"].append("Non European Union")
dictForFrame["Parent"].append("All countries of last or next residence")

dictForFrame["Label"].append("Europe exc European Union")
dictForFrame["Parent"].append("Non European Union")


# NonEu - Asia
dictForFrame["Label"].append("Asia")
dictForFrame["Parent"].append("Non European Union")

dictForFrame["Label"].append("Middle East and Central Asia")
dictForFrame["Parent"].append("Asia")
dictForFrame["Label"].append("East Asia")
dictForFrame["Parent"].append("Asia")
dictForFrame["Label"].append("South Asia")
dictForFrame["Parent"].append("Asia")
dictForFrame["Label"].append("South East Asia")
dictForFrame["Parent"].append("Asia")


# NonEu - Rest of World
dictForFrame["Label"].append("Rest of World")
dictForFrame["Parent"].append("Non European Union")

dictForFrame["Label"].append("Sub-Saharan Africa")
dictForFrame["Parent"].append("Rest of World")
dictForFrame["Label"].append("North Africa")
dictForFrame["Parent"].append("Rest of World")
dictForFrame["Label"].append("North America")
dictForFrame["Parent"].append("Rest of World")
dictForFrame["Label"].append("Central and South America")
dictForFrame["Parent"].append("Rest of World")
dictForFrame["Label"].append("Oceania")
dictForFrame["Parent"].append("Rest of World")


additionalsLabel = "Other Groupings"
if includeAdditionals:
    # Additional Groupings
    # i.e odd things, like trade organisations
    dictForFrame["Label"].append(additionalsLabel)
    dictForFrame["Parent"].append("All countries of last or next residence")

    dictForFrame["Label"].append("European Economic Area (EEA)")
    dictForFrame["Parent"].append(additionalsLabel)
    dictForFrame["Label"].append("European Free Trade Association (EFTA)")
    dictForFrame["Parent"].append(additionalsLabel)
    dictForFrame["Label"].append("Commonwealth")
    dictForFrame["Parent"].append(additionalsLabel)

    # Need to go somewhere
    # non-european-union-and-non-commonwealth-all
    # europe-inc-european-union
    # Neurope-exc-european-union
    dictForFrame["Parent"].append(additionalsLabel)
    dictForFrame["Label"].append("Non European Union and Non Commonwealth")
    dictForFrame["Parent"].append(additionalsLabel)
    dictForFrame["Label"].append("Europe inc European Union")
    
    # .... lastly, two child nodes for commonwealth
    dictForFrame["Label"].append("Old Commonwealth")
    dictForFrame["Parent"].append("Commonwealth")
    dictForFrame["Label"].append("New Commonwealth")
    dictForFrame["Parent"].append("Commonwealth")


# preview higher levels
pd.DataFrame.from_dict(dictForFrame)


,Label,Parent
0,All countries of last or next residence,
1,European Union,All countries of last or next residence
2,European Union EU15,European Union
3,European Union EU8,European Union
4,European Union EU2,European Union
5,Other European Union,European Union
6,Non European Union,All countries of last or next residence
7,Europe exc European Union,Non European Union
8,Asia,Non European Union
9,Middle East and Central Asia,Asia


## Extract the child nodes

In [66]:

inFile = "internationalmigrationtableofcontents.xls"
tab = loadxlstabs(inFile, ["Country of Residence Groupings"])[0]


Loading internationalmigrationtableofcontents.xls which has size 187904 bytes
Table names: ['Country of Residence Groupings']


## Sanity Checks

In [67]:
"""

making sure the principle groupings (i.e Europe, Asia etc - denoted as page-wde yellow wide merged rows) are in 
the order we expect, which is shown below:
"""

expectedItems = [
    "Country of last or next residence groupings in international migration tables",
    "Europe",
    
    "European Union (excluding United Kingdom)",
    "European Union EU15",
    "European Union EU8",
    "European Union EU2",
    "Other European Union",
    "Asia",
    "Middle East and Central Asia",
    "Rest of the World",
    "Sub-Saharan Africa",
    "Additional groupings available only in the IPS 4-Series tables ", 
    "European Economic Area (EEA)",
    "European Free Trade Association (EFTA)",
    "Commonwealth",
    "Old Commonwealth"
]

"""
the ._by_index(n) command can check the the selection where n is the NON_ZERO_INDEXED order.

i.e by_index(1) should be 'Country of last or next residence groupings in international migration tables'
    by_index(1) should be 'Europe'
    etc

Need to explicitly check this first as the logic falls apart otherwise.
"""

boldGroupings = tab.excel_ref("A").is_bold().is_not_blank().is_not_whitespace()

for i in range (0,len(expectedItems)):
    exp = expectedItems[i]
    got = boldGroupings.by_index(i+1).value
    assert exp == got, "Unexpected group formatting. Expected '{e}' but got '{g}'.".format(e=exp, g=got)
    
print("Checks passed. Bold cells in Columns A are where they're expected to be." )


Checks passed. Bold cells in Columns A are where they're expected to be.


## Functions

In [68]:
from pprint import pprint

# Gets the non-blank, doesnt-mention-migration values from between two specified unqiue-content cells
def sliceFromColumn(start, end):

    # Sanity check
    assert len(tab.filter(start)) == 1, "Aborting. '{s}' Should appear once.".format(s=start)
    
    if end != None:
        assert len(tab.filter(end)) == 1, "Aborting. '{e}' Should appear once.".format(e=end)

    # basic selection between bold start/end
    select = tab.filter(start).fill(DOWN).is_not_blank()
    
    # remove everything below a "Migrating before" cell, alsoeverything from the next bold value
    select = select - select.filter(contains_string("Migrating before")).expand(DOWN)
    
    # if extractting "Central and South America" need to remove some commoinwealth countries
    # they've changed the pattern here.
    if start == "Central and South America":
        
        # Check that the cell above Nigeria is empty (in case the ordering is changed)
        # iterate 1 item as bags cant be indexed...
        for cell in tab.excel_ref("D").filter("Nigeria").shift(UP):
            assert cell.value == "", "Aborting. Nigeria expected below a blank cell." + cell.value
        
        select = select - tab.excel_ref("D").filter("Nigeria").expand(DOWN)
    
    # If we have a specfied beneath section to remove, do so
    if end != None:
        remove = tab.filter(end).expand(DOWN)
        select = select - remove
    
    selectedVals = [x.value for x in select]

    # Remove notes about migration/migrating
    unwantedWords = ["migrated", "migration", "migrating"]
    for uw in unwantedWords:
        selectedVals = [x for x in selectedVals if uw not in x.lower()]

    return selectedVals


# Takes a list and a parent and adds to a dataframe
def appendListToFrameDict(parent, alist, dictForFrame):
    
    for country in alist:
        dictForFrame["Label"].append(country)
        dictForFrame["Parent"].append(parent)
        
    return dictForFrame
    

## Slicing Out Parents

This is based entireley on the use of BOLD in the cell formatting.

if you look at these example from sliceFromToList:
("European Union EU15", "European Union EU8"),

It's specifying that the children for "Europen Union EU15" are located below the bold cell value of
"European Union EU15" but above the bold cell value of "European Union EU8".

All functions only apply to a single column at a time.

In [69]:

"""
Slices all entries between two unique strings on the sheet.
If the second argument is 'None', we just select all below (that have text).
Where a 3rd item is in each tuple, it is the name we want to use in the hierarchy, otherwise its item[0]
"""
sliceFromToList = [
    ("European Union EU15", "European Union EU8"),
    ("European Union EU8", "European Union EU2"),
    ("European Union EU2", "Other European Union"),
    ("Other European Union", "Asia"),
    ("Other Europe", "East Asia", "Europe exc European Union"),
    ("Middle East and Central Asia", "Rest of the World"),
    ("East Asia","North Africa"),
    ("South Asia","North America"),
    ("South East Asia","Central and South America"),
    ("Sub-Saharan Africa", "Additional groupings available only in the IPS 4-Series tables"),
    ("North Africa",None),
    ("North America","New Commonwealth"),
    ("Central and South America", None),
    ("Oceania", None),
    ("Old Commonwealth", None)
]

for sliceFromTo in sliceFromToList:
    if len(sliceFromTo) == 2:
        outName = sliceFromTo[0]
    else:
        outName = sliceFromTo[2]
    selection = sliceFromColumn(sliceFromTo[0], sliceFromTo[1])
    dictForFrame = appendListToFrameDict(outName, selection, dictForFrame)


## Special Cases

Anything that doen't fit neatly into the extraction functions used above.

NOTE - most of these regroupings of already existing nodes (that already have
a parent).

In [70]:

# Non European Union and Non Commonwealth
# ---------------------------------------
# Have to selected everything then filter out

# IMPORTANT - skip the first 28 lines, we want child nodes only
df = pd.DataFrame.from_dict(dictForFrame)[28:]
countries = grouping["Label"].unique()

unwantedLabels = []
for unwanted in ["All countries of last or next residence", "European Union EU15", "European Union EU8", "European Union EU2", "Other European Union", "New Commonwealth", "Old Commonwealth"]:

    for label in df["Label"][df["Parent"] == unwanted].unique():
        unwantedLabels.append(label)
    
print(unwantedLabels)

for label in unwantedLabels:
    
    df = df.drop(df[df["Label"] == label].index)
    
print(df["Label"].unique())   
for label in df["Label"].unique():
    
    dictForFrame["Label"].append(label)
    dictForFrame["Parent"].append("Non European Union and Non Commonwealth")


['Belgium', 'Denmark', 'Finland', 'Aland Islands', 'France', 'Germany', 'Gibraltar', 'Greece', 'Republic of Ireland', 'Italy', 'Luxembourg', 'Netherlands', 'Portugal', 'Azores and Madeira', 'Spain and Balearic Islands', 'Canary Islands', 'Spain (not otherwise specified)', 'Sweden', 'Czech Republic', 'Estonia', 'Hungary', 'Latvia', 'Lithuania', 'Poland', 'Slovakia', 'Slovenia', 'Bulgaria', 'Romania', 'Cyprus (Ex)', 'Cyprus, Southern', 'Malta', 'Croatia', 'Australia', 'Heard Island and McDonald Islands', 'Canada', 'New Zealand', 'South Africa']
['Albania' 'Andorra' 'Armenia' 'Azerbaijan' 'Belarus' 'Bosnia Herzegovina'
 'Cyprus, Northern ' 'Czechoslovakia (Ex)' 'Faeroe Islands' 'Georgia'
 'Greenland' 'Iceland' 'Kosova' 'Liechtenstein' 'Macedonia' 'Moldova'
 'Monaco' 'Montenegro' 'Norway' 'Bouvet Island' 'Svalbard and Jan Mayen'
 'Russia' 'San Marino' 'Serbia' 'Switzerland' 'Turkey' 'Ukraine'
 'USSR (Ex)' 'Vatican City' 'Yugoslavia (Ex)'
 'Yugoslavia (not otherwise specified)' 'Afghanistan

In [71]:

# European Economic Area (EEA)
# ----------------------------

# this is European Union (Excluding United Kingdom).. i.e EU15, EU8 & EU2 
# plus three more:

if includeAdditionals:
    select = tab.excel_ref("A").filter("European Union (Excluding United Kingdom) as defined above plus:").fill(DOWN).is_not_blank()
    remove = tab.excel_ref("A").filter("European Free Trade Association (EFTA)").expand(DOWN)
    EEA = [x.value for x in (select - remove)]


    # Make a dataframe and filter to just EU15, EU8 and EU2, 
    # add each nation to our EEA list
    df = pd.DataFrame.from_dict(dictForFrame)

    for EU in ["European Union EU15", "European Union EU8", "European Union EU2"]:
        grouping = df[df["Parent"] == EU]
        for cell in grouping["Label"].unique():
            EEA.append(cell)

    for cell in EEA:

        dictForFrame["Label"].append(cell)
        dictForFrame["Parent"].append("European Economic Area (EEA)")


In [72]:

# European Free Trade Association (EFTA)
# --------------------------------------

if includeAdditionals:

    # as above + Switerzerland
    EEA.append("Switerzerland")

    for cell in EEA:

        dictForFrame["Label"].append(cell)
        dictForFrame["Parent"].append("European Free Trade Association (EFTA)")


In [73]:

# New Commonwealth
# ----------------
# needs to be handled sperarately as formatted across two columns for reasons

if includeAdditionals:
    anchor = tab.excel_ref("C").is_bold().filter("New Commonwealth")
    assert len(anchor) == 1, "There should only be one 'New Commonwealth' cell in column C."

    newCWselection = anchor.fill(DOWN).expand(RIGHT).is_not_blank().is_not_whitespace()
    remove = newCWselection.filter(("Migrating before 01/01/04:")).expand(DOWN)
    selection = newCWselection - remove

    for cell in selection:

        dictForFrame["Label"].append(cell.value)
        dictForFrame["Parent"].append("New Commonwealth")


In [74]:

# European Inc European Union
# ----------------------------

# Make a dataframe and filter to EU15, EU8, EU2 & Europe exc European Union
# add each nation to our EEA list
df = pd.DataFrame.from_dict(dictForFrame)
EIEU = []

for EU in ["European Union EU15", "European Union EU8", "European Union EU2", "Europe exc European Union"]:
        grouping = df[df["Parent"] == EU]
        for cell in grouping["Label"].unique():
            EIEU.append(cell)
            
for cell in EIEU:

    dictForFrame["Label"].append(cell)
    dictForFrame["Parent"].append("Europe Inc European Union")


## Populated Groups

These are the groups we've build so far. Each connected to by multiple hasData=True child nodes.'

In [75]:

# Sanity check our parent groups
print(pd.DataFrame.from_dict(dictForFrame)["Parent"].unique())

# and our source CSV so far (10 lines only)
pd.DataFrame.from_dict(dictForFrame)[:10]

['' 'All countries of last or next residence' 'European Union'
 'Non European Union' 'Asia' 'Rest of World' 'Other Groupings'
 'Commonwealth' 'European Union EU15' 'European Union EU8'
 'European Union EU2' 'Other European Union' 'Europe exc European Union'
 'Middle East and Central Asia' 'East Asia' 'South Asia' 'South East Asia'
 'Sub-Saharan Africa' 'North Africa' 'North America'
 'Central and South America' 'Oceania' 'Old Commonwealth'
 'Non European Union and Non Commonwealth' 'European Economic Area (EEA)'
 'European Free Trade Association (EFTA)' 'New Commonwealth'
 'Europe Inc European Union']


,Label,Parent
0,All countries of last or next residence,
1,European Union,All countries of last or next residence
2,European Union EU15,European Union
3,European Union EU8,European Union
4,European Union EU2,European Union
5,Other European Union,European Union
6,Non European Union,All countries of last or next residence
7,Europe exc European Union,Non European Union
8,Asia,Non European Union
9,Middle East and Central Asia,Asia


## Build the final CSV 

In [76]:

def codeListify(cell):
    cell = cell.replace("  ", " ")
    cell = cell.replace(" / ", " ")
    cell = cell.replace(" ", "-")
    cell = cell.replace("(", "")
    cell = cell.replace(")", "")
    cell = cell.replace(",", "")
    if cell[-1:] == "-":
        cell = cell[:-1]
    cell = cell.lower()
    return cell
    
    
# dataframe it
df = pd.DataFrame.from_dict(dictForFrame)

# apply codelist styling
df["Code"] = df["Label"].apply(codeListify)
df["ParentCode"] = df["Parent"].apply(codeListify)

# drop the old unstyle Parent Col, and order
finalDf = pd.DataFrame()
finalDf["Codelist"] = df["Code"] # just to populate number of rows, we'll override next
finalDf["Codelist"] = "migration-country"
finalDf["Code"] = df["Code"]
finalDf["Label"] = df["Label"]
finalDf["ParentCode"] = df["ParentCode"]



In [77]:

# Some last minute overides to make codes + extracted codes match

codeCorrections = [
    ["associationefta","association-efta"],
    ["areaeea","area-eea"],
    ["european-union15","european-union-eu15"],
    ["european-union8","european-union-eu8"],
    ["european-union2","european-union-eu2"],
    ["commonwealth-all","commonwealth"],
    ["old-commonwealth-all","old-commonwealth"],
    ["new-commonwealth-all","new-commonwealth"],
    ["spain-not-otherwise-specified", "spain"],
    ["usa", "united-states-of-america-usa"],
    ["non-european-union-and-non-commonwealth-all","non-european-union-and-non-commonwealth"],
    ["european-economic-areaeea", "european-economic-area-eea"]
]

for cr in codeCorrections:
    finalDf["Code"][finalDf["Code"] == cr[0]] = cr[1]
    finalDf["ParentCode"][finalDf["ParentCode"] == cr[0]] = cr[1]
    

In [78]:
finalDf.to_csv("Migration401_hierarchy.csv", index=False)

## Hierarchy

CREATE CONSTRAINT ON (n:`_generic_hierarchy_node_migration-country`) ASSERT n.code IS UNIQUE;

CREATE (node:`_generic_hierarchy_node_migration-country` { code:'country-of-residence-groupings',label:'Country of Residence Groupings' });

MATCH (parent:`_generic_hierarchy_node_migration-country` { code:'country-of-residence-groupings' })

WITH parent CREATE (node:`_generic_hierarchy_node_migration-country` { code:'asia',label:'Asia' })-[:hasParent]->(parent);

MATCH (parent:`_generic_hierarchy_node_migration-country` { code:'asia' })

In [79]:

with open("pyhierarchy.cypher", "w") as f:
    
    f.write("CREATE CONSTRAINT ON (n:`_generic_hierarchy_node_migration-country`) ASSERT n.code IS UNIQUE;")
    f.write("\n")
        
    f.write("CREATE (node:`_generic_hierarchy_node_migration-country` { code:'all-countries-of-last-or-next-residence',label:'All countries of last or next residence' });")
    f.write("\n")

    # Write the rows
    
    # Track whats being created - DONT create the same node twice
    alreadyDone = {}
    for i, row in finalDf.iterrows():
        
            # SKip first row, we've done that explicitly above
            if row["ParentCode"] != "":
                
                # If we've created a node with this code. Error if its not representing the same label. Otherwise skip. 
                if row["Code"] in alreadyDone.keys():
                    if row["Label"] != alreadyDone[row["Code"]]:
                        raise ValueError("The code {c} has labels of both: '{a}' and '{b}'.".format(a=row["Code"],b=row["Label"],c=alreadyDone[row["Code"]]))
                else:
                    # New node. Create it then add to alreadyDone dictionary.
                    f.write("CREATE (node:`_generic_hierarchy_node_migration-country` { code:'%s',label:'%s' });" % (row["Code"], row["Label"]))
                    f.write("\n")
                    alreadyDone.update({row["Code"]:row["Label"]})     
 
    # Write the relationships
    for i, row in finalDf.iterrows():
        
            if row["ParentCode"] != "":
                
                f.write("MATCH (parent:`_generic_hierarchy_node_migration-country`), (child:`_generic_hierarchy_node_migration-country`) where parent.code = \"%s\" and child.code = \"%s\" CREATE (child)-[:hasParent]->(parent);" %  (row["ParentCode"], row["Code"]))
                f.write("\n")
            